In [1]:
!pip install pretty_midi

     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
      --------------------------------------- 0.1/5.6 MB 403.5 kB/s eta 0:00:14
      --------------------------------------- 0.1/5.6 MB 409.6 kB/s eta 0:00:14
     - -------------------------------------- 0.1/5.6 MB 473.7 kB/s eta 0:00:12
     - -------------------------------------- 0.1/5.6 MB 473.7 kB/s eta 0:00:12
     - -------------------------------------- 0.1/5.6 MB 473.7 kB/s eta 0:00:12
     - -------------------------------------- 0.1/5.6 MB 473.7 kB/s eta 0:00:12
     - -------------------------------------- 0.1/5.6 MB 473.7 kB/s eta 0:00:12
     - -------------------------------------- 0.2/5.6 MB 443.5 kB/s eta 0:00:13
     

In [2]:
import tensorflow as tf

def generate_music(model, start_note, num_notes):
  """Generates a sequence of music notes using the LSTM model.

  Args:
    model: The LSTM model.
    start_note: The starting note.
    num_notes: The number of notes to generate.

  Returns:
    A list of music notes.
  """

  notes = [start_note]
  for _ in range(num_notes):
    prediction = model(notes)
    next_note = prediction.numpy().argmax()
    notes.append(next_note)

  return notes

def main():
  """The main function."""

  # Create the LSTM model.
  model = tf.keras.models.Sequential([
      tf.keras.layers.Embedding(128, 128),
      tf.keras.layers.LSTM(128),
      tf.keras.layers.Dense(128, activation="softmax")
  ])

  # Compile the model.
  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

  # Train the model.
  dataset = tf.data.Dataset.from_tensor_slices([(0, 60)])
  dataset = dataset.repeat(10000)
  dataset = dataset.batch(32)
  model.fit(dataset, epochs=100)

  # Save the model.
  model.save("model.h5")

if __name__ == "main":
  main()


In [3]:
import tensorflow as tf
import pretty_midi

def generate_midi(model, start_note, num_notes):
  """Generates a MIDI file from a sequence of music notes.

  Args:
    model: The LSTM model.
    start_note: The starting note.
    num_notes: The number of notes to generate.

  Returns:
    A MIDI file.
  """

  notes = generate_music(model, start_note, num_notes)

  # Create a PrettyMIDI object.
  midi = pretty_midi.PrettyMIDI()

  # Add notes to the PrettyMIDI object.
  for note in notes:
    pitch = note % 12 + 60
    velocity = 100
    duration = 0.5
    midi.add_note(pretty_midi.Note(pitch=pitch, velocity=velocity, start=0, end=duration))

  # Write the MIDI file to a file.
  midi.write("output.mid")

def main():
  """The main function."""

  # Load the LSTM model.
  model = tf.keras.models.load_model("model.h5")

  # Generate some music.
  start_note = 60 # C4
  num_notes = 100
  generate_midi(model, start_note, num_notes)

if __name__ == "main":
  main()
